# Index handling

Sometimes moving from layer to layer in a neural network involves rearranging the size from the output of the previous layer to the input of the next layer. This notebook demonstrates how to use this functionality in OMLT.

## Library Setup

Start by importing the libraries used in this project:

 - `numpy`: a general-purpose numerical library
 - `omlt`: the package this notebook demonstates.
 
We import the following classes from `omlt`:

 - `NetworkDefinition`: class that contains the nodes in a Neural Network
 - `InputLayer`, `DenseLayer`, `PoolingLayer2D`: the three types of layers used in this example
 - `IndexMapper`: used to reshape the data between layers

In [1]:
import numpy as np

from omlt.neuralnet import NetworkDefinition
from omlt.neuralnet.layer import DenseLayer, IndexMapper, InputLayer, PoolingLayer2D

Then we define a simple network that consists of a max pooling layer and a dense layer:

In [2]:
# define bounds for inputs
input_size = [9]
input_bounds = {}
for i in range(input_size[0]):
    input_bounds[(i)] = (-10.0, 10.0)

net = NetworkDefinition(scaled_input_bounds=input_bounds)

# define the input layer
input_layer = InputLayer(input_size)

net.add_layer(input_layer)

# define the pooling layer
input_index_mapper_1 = IndexMapper([9], [1, 3, 3])
maxpooling_layer = PoolingLayer2D(
    [1, 3, 3],
    [1, 2, 2],
    [2, 2],
    "max",
    [2, 2],
    1,
    input_index_mapper=input_index_mapper_1,
)

net.add_layer(maxpooling_layer)
net.add_edge(input_layer, maxpooling_layer)

# define the dense layer
input_index_mapper_2 = IndexMapper([1, 2, 2], [4])
dense_layer = DenseLayer(
    [4],
    [1],
    activation="linear",
    weights=np.ones([4, 1]),
    biases=np.zeros(1),
    input_index_mapper=input_index_mapper_2,
)

net.add_layer(dense_layer)
net.add_edge(maxpooling_layer, dense_layer)

for layer_id, layer in enumerate(net.layers):
    print(f"{layer_id}\t{layer}")

0	InputLayer(input_size=[9], output_size=[9])
1	PoolingLayer(input_size=[1, 3, 3], output_size=[1, 2, 2], strides=[2, 2], kernel_shape=[2, 2]), pool_func_name=max
2	DenseLayer(input_size=[4], output_size=[1])


In this example, `input_index_mapper_1` maps outputs of `input_layer` (with size [9])  to the inputs of `maxpooling_layer` (with size [1, 3, 3]), `input_index_mapper_2` maps the outputs of `maxpooling_layer` (with size [1, 2, 2]) to the inputs of `dense_layer` (with size [4]). Given an input, we can evaluate each layer to see how `IndexMapper` works: 

In [3]:
x = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
y1 = maxpooling_layer.eval_single_layer(x)
print("outputs of maxpooling_layer:\n", y1)
y2 = dense_layer.eval_single_layer(y1)
print("outputs of dense_layer:\n", y2)

outputs of maxpooling_layer:
 [[[5. 6.]
  [8. 9.]]]
outputs of dense_layer:
 [28.]


Without `IndexMapper`, the output of `maxpooling_layer` is identical to the input of `dense_layer`. When using `IndexMapper`, using `input_indexes_with_input_layer_indexes` can provide the mapping between indexes. Therefore, there is no need to define variables for the inputs of each layer (except for `input_layer`). As shown in the following, we print both input indexes and output indexes for each layer. Also, we give the mapping between indexes of two adjacent layers, where `local_index` corresponds to the input indexes of the current layer and `input_index` corresponds to the output indexes of previous layer.

In [4]:
print("input indexes of input_layer:")
print(input_layer.input_indexes)
print("output indexes of input_layer:")
print(input_layer.output_indexes)

input indexes of input_layer:
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,)]
output indexes of input_layer:
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,)]


In [5]:
print("input indexes of maxpooling_layer:")
print(maxpooling_layer.input_indexes)
print("output indexes of maxpooling_layer:")
print(maxpooling_layer.output_indexes)
print("input_index_mapping_1:")
for local_index, input_index in maxpooling_layer.input_indexes_with_input_layer_indexes:
    print("local_index:", local_index, "input_index:", input_index)

input indexes of maxpooling_layer:
[(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 2, 0), (0, 2, 1), (0, 2, 2)]
output indexes of maxpooling_layer:
[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1)]
input_index_mapping_1:
local_index: (0, 0, 0) input_index: (0,)
local_index: (0, 0, 1) input_index: (1,)
local_index: (0, 0, 2) input_index: (2,)
local_index: (0, 1, 0) input_index: (3,)
local_index: (0, 1, 1) input_index: (4,)
local_index: (0, 1, 2) input_index: (5,)
local_index: (0, 2, 0) input_index: (6,)
local_index: (0, 2, 1) input_index: (7,)
local_index: (0, 2, 2) input_index: (8,)


In [6]:
print("input indexes of dense_layer:")
print(dense_layer.input_indexes)
print("output indexes of dense_layer:")
print(dense_layer.output_indexes)
print("input_index_mapping_2:")
for local_index, input_index in dense_layer.input_indexes_with_input_layer_indexes:
    print("local_index:", local_index, "input_index:", input_index)

input indexes of dense_layer:
[(0,), (1,), (2,), (3,)]
output indexes of dense_layer:
[(0,)]
input_index_mapping_2:
local_index: (0,) input_index: (0, 0, 0)
local_index: (1,) input_index: (0, 0, 1)
local_index: (2,) input_index: (0, 1, 0)
local_index: (3,) input_index: (0, 1, 1)
